In [ ]:
import geopandas as gpd
import momepy
from libpysal import graph
import pandas as pd
import numpy as np
import shapely
from scipy import spatial

## Load data

In [ ]:
# load morphometric elements of a site
buildings = gpd.read_file(r"preprocessed_data\berlin_buildings_checked.gpkg")
streets = gpd.read_file(r"preprocessed_data\berlin_streets_simplified.gpkg")
water_polygons = gpd.read_file(r"overture_data\berlin_waterbodies.gpkg")
water_lines = gpd.read_file(r"preprocessed_data\berlin_waterlines_checked.gpkg")

# buildings = gpd.read_file(r"preprocessed_data\hongkong_buildings_checked.gpkg")
# streets = gpd.read_file(r"preprocessed_data\hongkong_streets_simplified.gpkg")
# water_polygons = gpd.read_file(r"overture_data\hongkong_waterbodies.gpkg")
# water_lines = gpd.read_file(r"preprocessed_data\hongkong_waterlines_checked.gpkg")

#buildings = gpd.read_file(r"preprocessed_data\paris_buildings_checked.gpkg")
# streets = gpd.read_file(r"preprocessed_data\paris_streets_simplified.gpkg")
# water_polygons = gpd.read_file(r"overture_data\paris_waterbodies.gpkg")
# water_lines = gpd.read_file(r"preprocessed_data\paris_waterlines_checked.gpkg")

# buildings = gpd.read_file(r"preprocessed_data\rome_buildings_checked.gpkg")
# streets = gpd.read_file(r"preprocessed_data\rome_streets_simplified.gpkg")
# water_polygons = gpd.read_file(r"overture_data\rome_waterbodies.gpkg")
# water_lines = gpd.read_file(r"preprocessed_data\rome_waterlines_checked.gpkg")

# buildings = gpd.read_file(r"preprocessed_data\saopaulo_buildings_checked.gpkg")
# streets = gpd.read_file(r"preprocessed_data\saopaulo_streets_simplified.gpkg")
# water_polygons = gpd.read_file(r"overture_data\saopaulo_waterbodies.gpkg")
# water_lines = gpd.read_file(r"preprocessed_data\saopaulo_waterlines_checked.gpkg")

## Checks

Before running the enclosed tessellation ensure that there are no duplicate buildings. If there are, drop them. Otherwise the tessellation crashes.

In [ ]:
duplicates = buildings[buildings.geometry.duplicated(keep=False)]
print(duplicates)

In [ ]:
duplicates = buildings[buildings.geometry.centroid.duplicated(keep=False)]
print(duplicates)

In [ ]:
buildings.index.duplicated().any()

In [ ]:
buildings.index.is_unique

In [ ]:
# number of unique coordinates should be equal to the number of all coordinates
coordinates = shapely.get_coordinates(buildings.centroid)
coordinates = np.round(coordinates, 6)
unique = np.unique(coordinates, axis=0)
print(unique.shape, coordinates.shape)

Check also for collinear redundant building centroids. If such exist, drop those buildings. Otherwise the tessellation crashes.

In [ ]:
coordinates = shapely.get_coordinates(buildings.centroid)
dt = spatial.Delaunay(coordinates)
dt.coplanar

In [ ]:
if dt.coplanar.size > 0:
    buildings = buildings.drop(buildings.index[dt.coplanar[:, 0]])
    buildings = buildings.reset_index(drop=True)
    
    print(f"Removed {len(dt.coplanar[:, 0])} redundant buildings.")
else:
    print("No redundant buildings found.")

Check for collapsing features in tessellation. If exist, drop them.

In [ ]:
check = momepy.CheckTessellationInput(buildings)

In [ ]:
buildings = buildings.drop(check.collapse.index)
buildings = buildings.reset_index(drop=True)

## Enclosed tessellation

In [ ]:
limit = momepy.buffered_limit(buildings, buffer="adaptive")

In [ ]:
# defining streets, waterbodies and waterlines as enclosures
enclosures = momepy.enclosures(
    streets,
    limit=limit,
    additional_barriers=[water_polygons.boundary, water_lines],
)

In [ ]:
enclosed_tess = momepy.enclosed_tessellation(buildings, enclosures)

In [ ]:
# drop duplicate tessellation cells if exist
enclosed_tess = enclosed_tess.drop(enclosed_tess.index[enclosed_tess.index.duplicated()])

In [ ]:
# add building index as a new column
enclosed_tess["building_index"] = enclosed_tess.index

In [ ]:
# drop cells without building
enclosed_tess = enclosed_tess[enclosed_tess["building_index"]>=0]

In [ ]:
# save tessellation cells
enclosed_tess.to_file(r'momepy\berlin_etc.gpkg', driver="GPKG")
# enclosed_tess.to_file(r'momepy\hongkong_etc.gpkg', driver="GPKG")
# enclosed_tess.to_file(r'momepy\paris_etc.gpkg', driver="GPKG")
# enclosed_tess.to_file(r'momepy\rome_etc.gpkg', driver="GPKG")
# enclosed_tess.to_file(r'momepy\saopaulo_etc.gpkg', driver="GPKG")

## Connectivity (Street Network Analysis)

In [ ]:
# assign street ID to nearest building and cell
buildings["street_index"] = momepy.get_nearest_street(buildings, streets)
enclosed_tess["street_index"] = buildings["street_index"]

In [ ]:
# generate graph
graph = momepy.gdf_to_nx(streets)

In [ ]:
# clustering
graph = momepy.clustering(graph, name="clustering")

# node degree
graph = momepy.node_degree(graph, name='degree')

# mean node degree
graph = momepy.mean_node_degree(graph, radius=5, name='mean_nd')
graph = momepy.mean_node_degree(graph, radius=400, distance='mm_len', name='mean_nd400')

# length of cul-de-sacs
graph = momepy.cds_length(graph, radius=5, mode='sum', name='cds_len', degree='degree', length='mm_len', distance=None)
graph = momepy.cds_length(graph, radius=400, mode='sum', name='cds_len400', degree='degree', length='mm_len', distance='mm_len')

# mean node distance
graph = momepy.mean_node_dist(graph, name='meanlen', length='mm_len')

# node density
graph = momepy.node_density(graph, radius=5, length='mm_len')
graph = momepy.node_density(graph, radius=400, length='mm_len', distance='mm_len')

#cyclomatic complexity
graph = momepy.cyclomatic(graph, radius=5, name='cyclomatic')
graph = momepy.cyclomatic(graph, radius=400, distance="mm_len", name='cyclomatic400')

# edge to node ratio
graph = momepy.edge_node_ratio(graph, radius=5, name='edge_node_ratio')
graph = momepy.edge_node_ratio(graph, radius=400, distance="mm_len", name='edge_node_ratio400')

# gamma connectivity index
graph = momepy.gamma(graph, radius=5, name='gamma')
graph = momepy.gamma(graph, radius=400, distance="mm_len", name='gamma400')

# meshedness
graph = momepy.meshedness(graph, name="meshedness", radius=5)
graph = momepy.meshedness(graph, radius=400, name="meshedness400", distance="mm_len")

In [ ]:
# assign edge and node ID to nearest building and cell
buildings["edge_index"] = momepy.get_nearest_street(buildings, edges)
buildings["node_index"] = momepy.get_nearest_node(buildings, nodes, edges, buildings["edge_index"])

enclosed_tess["edge_index"] = buildings["edge_index"]
enclosed_tess["node_index"] = buildings["node_index"]

In [ ]:
del graph

In [ ]:
from libpysal import graph

## Dimension

In [ ]:
# street length
streets['length'] = streets.length

# width, openness, width deviation of a street
street_profile = momepy.street_profile(streets, buildings)

# area (building, tessellation cell)
buildings['building_area'] = buildings.area
enclosed_tess['tess_area'] = enclosed_tess.area

# building coverage area ratio
enclosed_tess["t_coverage_area_ratio"] = buildings.area / enclosed_tess.area

# longest axis length (building, tessellation cell)
buildings["b_axis_length"] = momepy.longest_axis_length(buildings)
enclosed_tess["t_axis_length"] = momepy.longest_axis_length(enclosed_tess)

# perimeter wall length
buildings_queen = graph.Graph.build_contiguity(buildings, rook=False, strict=False)
buildings['b_perimeter_wall_length'] = momepy.perimeter_wall(buildings, buildings_queen, buffer=0.1)

# courtyard area of a building
buildings['b_courtyard_area'] = momepy.courtyard_area(buildings)

## Shape

In [ ]:
# circular compactness (building, tessellation cell)
buildings["b_circular_compactness"] = momepy.circular_compactness(buildings)
enclosed_tess["t_circular_compactness"] = momepy.circular_compactness(enclosed_tess)

# compactness-weighted axis (building, tessellation cell)
buildings["b_compactness_weighted_axis"] = momepy.compactness_weighted_axis(buildings)
enclosed_tess["t_compactness_weighted_axis"] = momepy.compactness_weighted_axis(enclosed_tess)

# convexity (building, tessellation cell)
buildings["b_convexity"] = momepy.convexity(buildings)
enclosed_tess["t_convexity"] = momepy.convexity(enclosed_tess)

# elongation (building, tessellation cell)
buildings["b_elongation"] = momepy.elongation(buildings)
enclosed_tess["t_elongation"] = momepy.elongation(enclosed_tess)

#equivalent rectangular index (building, tessellation cell)
buildings["b_equivalent_rectangular_index"] = momepy.equivalent_rectangular_index(buildings)
enclosed_tess["t_equivalent_rectangular_index"] = momepy.equivalent_rectangular_index(enclosed_tess)

# facade ratio (building, tessellation cell)
buildings["b_facade_ratio"] = momepy.facade_ratio(buildings)
enclosed_tess["t_facade_ratio"] = momepy.facade_ratio(enclosed_tess)

# fractal dimension (building, tessellation cell)
buildings["b_fractal_dimension"] = momepy.fractal_dimension(buildings)
enclosed_tess["t_fractal_dimension"] = momepy.fractal_dimension(enclosed_tess)

# rectangularity (building, tessellation cell)
buildings["b_rectangularity"] = momepy.rectangularity(buildings)
enclosed_tess["t_rectangularity"] = momepy.rectangularity(enclosed_tess)

# shape index (building, tessellation cell)
buildings["b_shape_index"] = momepy.shape_index(buildings)
enclosed_tess["t_shape_index"] = momepy.shape_index(enclosed_tess)

# square compactness (building, tessellation cell)
buildings["b_square_compactness"] = momepy.square_compactness(buildings)
enclosed_tess["t_square_compactness"] = momepy.square_compactness(enclosed_tess)

# courtyard index
buildings["b_courtyard_index"] = momepy.courtyard_index(buildings, courtyard_area=buildings['b_courtyard_area'])

# street linearity
streets['s_linearity']  = momepy.linearity(streets)

## Spatial Distribution

In [ ]:
## tessellation contiguity graphs
# tessellation cells within 1 topological step
tess_fuzzy_1 = graph.Graph.build_fuzzy_contiguity(enclosed_tess, buffer=1e-4)

# tessellation cells within 2 topological steps
tess_fuzzy_2 = tess_fuzzy_1.higher_order(2)

# tessellation cells within 3 topological steps
tess_fuzzy_3 = tess_fuzzy_1.higher_order(3)

In [ ]:
# number of neighboring tessellation cells within 1, 2, 3 topological steps
enclosed_tess["t_neighbours_order1"] = momepy.neighbors(enclosed_tess, tess_fuzzy_1)
enclosed_tess["t_neighbours_order2"] = momepy.neighbors(enclosed_tess, tess_fuzzy_2)
enclosed_tess["t_neighbours_order3"] = momepy.neighbors(enclosed_tess, tess_fuzzy_3)

In [ ]:
# mean distance to neighboring tessellation cells within 2, 3 topological steps
enclosed_tess["t_neighbours_dist_order2"] = momepy.neighbor_distance(enclosed_tess, tess_fuzzy_2)
enclosed_tess["t_neighbours_dist_order3"] = momepy.neighbor_distance(enclosed_tess, tess_fuzzy_3)

In [ ]:
# neighborhood description (granularity)
enclosed_tess["granularity"] = tess_fuzzy_1.describe(enclosed_tess.area)["sum"]

In [ ]:
## buildings distance graphs
# buildings within 20 m
dist20 = graph.Graph.build_distance_band(buildings.centroid, threshold=20)

# buildings within 100 m
dist100 = graph.Graph.build_distance_band(buildings.centroid, threshold=100)

# buildings within 200 m
dist200 = graph.Graph.build_distance_band(buildings.centroid, threshold=200)

In [ ]:
# number of neighboring buildings within 20, 100, 200 m
buildings["b_neighbours_50m"] = momepy.neighbors(buildings, dist20)
buildings["b_neighbours_100m"] = momepy.neighbors(buildings, dist100)
buildings["b_neighbours_200m"] = momepy.neighbors(buildings, dist200)

# mean distance to neighboring buildings within 20, 100, 200 m
buildings["b_neighbours_dist_20m"] = momepy.neighbor_distance(buildings, dist20)
buildings["b_neighbours_dist_100m"] = momepy.neighbor_distance(buildings, dist100)
buildings["b_neighbours_dist_200m"] = momepy.neighbor_distance(buildings, dist200)

In [ ]:
## building K-nearest neighbors graphs, k=[10,20,30]
knn10 = graph.Graph.build_knn(buildings.centroid, k=10)
knn20 = graph.Graph.build_knn(buildings.centroid, k=20)
knn30 = graph.Graph.build_knn(buildings.centroid, k=30)

In [ ]:
# mean distance to 10, 20, 30 neighboring buildings
buildings["b_neighbours_dist_k10"] = momepy.neighbor_distance(buildings, knn10)
buildings["b_neighbours_dist_k20"] = momepy.neighbor_distance(buildings, knn20)
buildings["b_neighbours_dist_k30"] = momepy.neighbor_distance(buildings, knn30)

In [ ]:
## level of building adjacency within 200 m neighborhood
# building contiguity graph
buildings_rook = graph.Graph.build_contiguity(buildings, rook=True)

# buildings within 200 m (neighborhood)
dist200 = graph.Graph.build_distance_band(buildings.centroid, 200)

buildings["building_adjacency"] = momepy.building_adjacency(
    contiguity_graph=buildings_rook, neighborhood_graph=dist200)

In [ ]:
## mean interbuilding distance within 200 m neighborhood
tess_fuzzy_1_building_subgraph = tess_fuzzy_1.subgraph(buildings.index.tolist())
dist200 = graph.Graph.build_distance_band(buildings.centroid, 200)

buildings["mean_ib_dist"] = momepy.mean_interbuilding_distance(
    buildings, adjacency_graph=tess_fuzzy_1_building_subgraph, neighborhood_graph=dist200)

## if above not working, try this:
# missing_split = check.split.loc[check.split.index.difference(enclosed_tess.index)]
# missing_collapse = check.collapse.loc[check.collapse.index.difference(enclosed_tess.index)]
# missing = missing_split.index.union(missing_collapse.index)

# filtered_idxs = [idx for idx in buildings.index if idx not in missing.tolist()]

# tess_fuzzy_1_building_subgraph = tess_fuzzy_1.subgraph(filtered_idxs)

# buildings_filtered = buildings.drop(missing)
# dist200 = graph.Graph.build_distance_band(buildings_filtered.centroid, 200)

# buildings["mean_ib_dist"] = momepy.mean_interbuilding_distance(
#     buildings_filtered, adjacency_graph=tess_fuzzy_1_building_subgraph, neighborhood_graph=dist200)

In [ ]:
# the length of shared walls of adjacent buildings
buildings["shared_walls"] = momepy.shared_walls(buildings, strict=False, tolerance=0.01) / buildings.length

# building street alignment
buildings['b_street_alignment'] = momepy.street_alignment(
    momepy.orientation(buildings), momepy.orientation(streets), buildings["street_index"])

## Weighted dimension and shape characters

### Building characters

Building dimension and shape characters weighted by areas of building in 100 and 200 m neighborhood

In [ ]:
# building axis length weighted
buildings["b_axis_length_weighted_100m"] = momepy.weighted_character(
    buildings.b_axis_length, buildings.area, dist100)

buildings["b_axis_length_weighted_200m"] = momepy.weighted_character(
    buildings.b_axis_length, buildings.area, dist200)

# building perimeter wall length weighted
buildings["b_perimeter_wall_length_weighted_100m"] = momepy.weighted_character(
    buildings.b_perimeter_wall_length, buildings.area, dist100)

buildings["b_perimeter_wall_length_weighted_200m"] = momepy.weighted_character(
    buildings.b_perimeter_wall_length, buildings.area, dist200)

# building circular compactness weighted
buildings["b_circular_compactness_weighted_100m"] = momepy.weighted_character(
    buildings.b_circular_compactness, buildings.area, dist100)

buildings["b_circular_compactness_weighted_200m"] = momepy.weighted_character(
    buildings.b_circular_compactness, buildings.area, dist200)

# building compactness-weighted axis weighted
buildings["b_compactness_weighted_axis_weighted_100m"] = momepy.weighted_character(
    buildings.b_compactness_weighted_axis, buildings.area, dist100)

buildings["b_compactness_weighted_axis_weighted_200m"] = momepy.weighted_character(
    buildings.b_compactness_weighted_axis, buildings.area, dist200)

# building convexity weighted
buildings["b_convexity_weighted_100m"] = momepy.weighted_character(
    buildings.b_convexity, buildings.area, dist100)

buildings["b_convexity_weighted_200m"] = momepy.weighted_character(
    buildings.b_convexity, buildings.area, dist200)

# building elongation weighted
buildings["b_elongation_weighted_100m"] = momepy.weighted_character(
    buildings.b_elongation, buildings.area, dist100)

buildings["b_elongation_weighted_200m"] = momepy.weighted_character(
    buildings.b_elongation, buildings.area, dist200)

# building equivalent rectangular index weighted
buildings["b_equivalent_rectangular_index_weighted_100m"] = momepy.weighted_character(
    buildings.b_equivalent_rectangular_index, buildings.area, dist100)

buildings["b_equivalent_rectangular_index_weighted_200m"] = momepy.weighted_character(
    buildings.b_equivalent_rectangular_index, buildings.area, dist200)

# building facade ratio weighted
buildings["b_facade_ratio_weighted_100m"] = momepy.weighted_character(
    buildings.b_facade_ratio, buildings.area, dist100)

buildings["b_facade_ratio_weighted_200m"] = momepy.weighted_character(
    buildings.b_facade_ratio, buildings.area, dist200)

# building fractal dimension weighted
buildings["b_fractal_dimension_weighted_100m"] = momepy.weighted_character(
    buildings.b_fractal_dimension, buildings.area, dist100)

buildings["b_fractal_dimension_weighted_200m"] = momepy.weighted_character(
    buildings.b_fractal_dimension, buildings.area, dist200)

# building rectangularity weighted
buildings["b_rectangularity_weighted_100m"] = momepy.weighted_character(
    buildings.b_rectangularity, buildings.area, dist100)

buildings["b_rectangularity_weighted_200m"] = momepy.weighted_character(
    buildings.b_rectangularity, buildings.area, dist200)

# building shape index weighted
buildings["b_shape_index_weighted_100m"] = momepy.weighted_character(
    buildings.b_shape_index, buildings.area, dist100)

buildings["b_shape_index_weighted_200m"] = momepy.weighted_character(
    buildings.b_shape_index, buildings.area, dist200)

# building square compactness weighted
buildings["b_square_compactness_weighted_100m"] = momepy.weighted_character(
    buildings.b_square_compactness, buildings.area, dist100)

buildings["b_square_compactness_weighted_200m"] = momepy.weighted_character(
    buildings.b_square_compactness, buildings.area, dist200)

# building street alignment weighted
buildings["b_street_alignment_weighted_100m"] = momepy.weighted_character(
    buildings.b_street_alignment, buildings.area, dist100)

buildings["b_street_alignment_weighted_200m"] = momepy.weighted_character(
    buildings.b_street_alignment, buildings.area, dist200)

### Tessellation cell characters

Tessellation cell dimension and shape characters weighted by areas of cells within 3 topological steps

In [ ]:
# tessellation axis length weighted
enclosed_tess["t_axis_length_weighted_order3"] = momepy.weighted_character(
    enclosed_tess.t_axis_length, enclosed_tess.area, tess_fuzzy_3)

# building coverage area ratio weighted
enclosed_tess["t_coverage_area_ratio_weighted_order3"] = momepy.weighted_character(
    enclosed_tess.t_coverage_area_ratio, enclosed_tess.area, tess_fuzzy_3)

# tessellation circular compactness weighted
enclosed_tess["t_circular_compactness_weighted_order3"] = momepy.weighted_character(
    enclosed_tess.t_circular_compactness, enclosed_tess.area, tess_fuzzy_3)

# tessellation compactness-weighted axis weighted
enclosed_tess["t_compactness_weighted_axis_weighted_order3"] = momepy.weighted_character(
    enclosed_tess.t_compactness_weighted_axis, enclosed_tess.area, tess_fuzzy_3)

# tessellation convexity weighted
enclosed_tess["t_convexity_weighted_order3"] = momepy.weighted_character(
    enclosed_tess.t_convexity, enclosed_tess.area, tess_fuzzy_3)

# tessellation elongation weighted
enclosed_tess["t_elongation_weighted_order3"] = momepy.weighted_character(
    enclosed_tess.t_elongation, enclosed_tess.area, tess_fuzzy_3)

# tessellation equivalent rectangular index weighted
enclosed_tess["t_equivalent_rectangular_index_weighted_order3"] = momepy.weighted_character(
    enclosed_tess.t_equivalent_rectangular_index, enclosed_tess.area, tess_fuzzy_3)

# tessellation facade area ratio weighted
enclosed_tess["t_facade_ratio_weighted_order3"] = momepy.weighted_character(
    enclosed_tess.t_facade_ratio, enclosed_tess.area, tess_fuzzy_3)

# tessellation fractal dimension weighted
enclosed_tess["t_fractal_dimension_weighted_order3"] = momepy.weighted_character(
    enclosed_tess.t_fractal_dimension, enclosed_tess.area, tess_fuzzy_3)

# tessellation rectangularity weighted
enclosed_tess["t_rectangularity_weighted_order3"] = momepy.weighted_character(
    enclosed_tess.t_rectangularity, enclosed_tess.area, tess_fuzzy_3)

# tessellation shape index weighted
enclosed_tess["t_shape_index_weighted_order3"] = momepy.weighted_character(
    enclosed_tess.t_shape_index, enclosed_tess.area, tess_fuzzy_3)

# tessellation square compactness weighted
enclosed_tess["t_square_compactness_weighted_order3"] = momepy.weighted_character(
    enclosed_tess.t_square_compactness, enclosed_tess.area, tess_fuzzy_3)

## Diversity

For each primary character, three contextual characters were derived by calculating the 25th, 50th, and 75th percentile of its values within the set of tessellation cells located within three topological steps from the target tessellation cell.

In [ ]:
# join building characters to tessellation cells
building_attr_keep = buildings.drop(columns=['building_index', 'geometry', 'street_index'])
enclosed_tess[building_attr_keep.columns] = building_attr_keep

# join streets characters to tessellation cells
street_attr_keep = streets.drop(columns=['geometry'])
enclosed_tess = enclosed_tess.merge(
    street_attr_keep,
    left_on="street_index",
    right_index=True,
    how="left",
)

# join nodes and edges characters to tessellation cells
enclosed_tess = enclosed_tess.merge(
    nodes.drop(columns="geometry"),
    left_on="node_index",
    right_index=True,
    how="left",
)

enclosed_tess = enclosed_tess.merge(
    edges.drop(columns="geometry"),
    left_on="edge_index",
    right_index=True,
    how="left",
)

In [ ]:
# cleaning
enclosed_tess = enclosed_tess.drop(columns=['mm_len','cdsbool','node_start','node_end','x','y','nodeID'])

In [ ]:
# 25th, 50th, and 75th percentile of character value within cells located within 3 topological steps from the target cell
percentiles = []
for column in enclosed_tess.columns.drop(
    [
        "street_index",
        "enclosure_index",
        "building_index",
        "geometry",
        "edge_index",
        "node_index"
    ]
):
    # calculate percentiles within 3 topological steps, return 3 values
    perc = momepy.percentile(enclosed_tess[column], tess_fuzzy_3)
    
    # add character name
    perc.columns = [f"{column}_" + str(x) for x in perc.columns]
    
    percentiles.append(perc)

percentiles_df = pd.concat(percentiles, axis=1)

In [ ]:
# add cells geometry
percentiles_df['geometry'] = enclosed_tess['geometry']
percentiles_gdf = gpd.GeoDataFrame(percentiles_df, geometry='geometry')

In [ ]:
# save
percentiles_gdf.to_parquet(r'momepy\berlin_morphometrics.parquet', compression='zstd')
#percentiles_gdf.to_parquet(r'momepy\hongkong_morphometrics.parquet', compression='zstd')
#percentiles_gdf.to_parquet(r'momepy\paris_morphometrics.parquet', compression='zstd')
#percentiles_gdf.to_parquet(r'momepy\rome_morphometrics.parquet', compression='zstd')
#percentiles_gdf.to_parquet(r'momepy\saopaulo_morphometrics.parquet', compression='zstd')